In [8]:
import requests
import csv
from bs4 import BeautifulSoup

# Base URL for restaurant listings
BASE_URL = 'https://www.yellowpages-uae.com/uae/restaurant'

# Function to extract business details from a given URL
def extract_business_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # List to store extracted business data
    businesses = []

    # Iterate over each business entry on the page
    for business in soup.find_all('div', {'class': 'row box foc'}):
        # Extract individual business details
        name = business.find('h2', {'class': 'cmp_name'})
        location = business.find('span', itemprop='streetAddress')
        city = business.find('strong', itemprop='addressLocality')
        po_box = business.find('span', itemprop='postalCode')

        phone = business.find('span', itemprop='telephone')
        phone_text = phone.text.strip() if phone else ''

        mobiles = business.find_all('span', itemprop='telephone')
        mobile_text = mobiles[1].text.strip() if len(mobiles) > 1 else ''

        link = business.find('a')
        logo = business.find('img', itemprop='image')

        # Create a dictionary to store business data
        business_data = {
            'Name': name.text.strip() if name else '',
            'Location': location.text.strip() if location else '',
            'City': city.text.strip() if city else '',
            'P.O Box': po_box.text.strip() if po_box else '',
            'Phone': phone_text,
            'Mobile': mobile_text,
            'Company Page Link': BASE_URL + link['href'] if link else '',
            'Logo URL': logo.get('data-src', '') if logo and 'data-src' in logo.attrs else ''
        }

        # Append the business data to the list
        businesses.append(business_data)

    return businesses

# Function to scrape all pages of restaurant listings
def scrape_all_pages(base_url):
    all_businesses = []
    page_number = 1

    # Loop through each page to extract businesses
    while True:
        url = f'{base_url}-{page_number}.html'
        businesses = extract_business_data(url)

        # Break if no more businesses are found
        if not businesses:
            break

        all_businesses.extend(businesses)
        page_number += 1

    return all_businesses

# Function to save data to a CSV file
def save_to_csv(data, filename='results.csv'):
    with open(filename, 'w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=data[0].keys())
        writer.writeheader()
        writer.writerows(data)

# Main execution
if __name__ == '__main__':
    all_businesses = scrape_all_pages(BASE_URL)
    save_to_csv(all_businesses)

    # Print the number of businesses scraped
    print(f'Scraped businesses and saved to results.csv')

Scraped businesses and saved to results.csv
